In [ ]:
!pip install matplotlib

In [1]:
import requests
import time
import sys
import json
import pickle
import numpy as np
import pandas as pd
from google.colab import userdata
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# request headers, only need to refresh api key
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en,pl-PL;q=0.9,pl;q=0.8,en-US;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": userdata.get("RIOT_API")
}
headers

### Getting Match Ids from the Riot API

In [ ]:
ranks = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "EMERALD", "DIAMOND", "MASTER", "GRANDMASTER", "CHALLENGER"]

In [ ]:
#Rank Distirbution LOL to get games in correct distribution
total_games = 100000

rank_percentages = {
    "IRON": 7.5,
    "BRONZE": 20,
    "SILVER": 19,
    "GOLD": 19,
    "PLATINUM": 19,
    "EMERALD": 13,
    "DIAMOND": 3,
    "MASTER": 0.5,
    "GRANDMASTER": 0.04,
    "CHALLENGER": 0.02
}

# Calculate the number of games for each rank
rank_games = {rank: int(total_games * percentage / 100) for rank, percentage in rank_percentages.items()}

In [ ]:
def getPuuids(summonerIds):
  puuids = []
  for summonerId in summonerIds:
    url_summoners = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}"
    r = requests.get(url=url_summoners, headers=headers)
    if r.status_code != 200:
      print(r.status_code)
      if r.status_code == 429:
        #time.sleep(r.headers["Retry-After"])
        #print(f"Hit limit - proceeding in {r.headers['Retry-After']}")
        print("LIMIT")
      continue
    puuids.append(r.json()["puuid"])
    time.sleep(0.8)
  return puuids

In [ ]:
def getMatchIds(rank, numOfGames):
  url = f"https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{rank}/I?page=5"
  r = requests.get(url=url, headers=headers)
  players = r.json()
  summonerIds = [player["summonerId"] for player in players]
  puuids = getPuuids(summonerIds)
  totalGames = 0
  match_ids = []
  for puuid in puuids:
    if totalGames > numOfGames:
      break
    url4 = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=20"
    r = requests.get(url=url4, headers=headers)
    for id in r.json():
      match_ids.append(id)
      print(id)
    totalGames+=20
    time.sleep(0.8)
  return match_ids

In [ ]:
allMatchIds = []
for rank in ranks:
  allMatchIds.append(getMatchIds(rank, rank_games[rank]))

In [ ]:
matchIds = [item for sublist in allMatchIds for item in sublist]

In [ ]:
to_save = {"match_ids": np.unique(matchIds.flatten())}
with open("/content/drive/MyDrive/ML/match_ids_1_unique.json", "wb") as f:
  pickle.dump(to_save, f)

### Getting Match Data from the Match Ids using the Riot API

In [ ]:
with open("/content/drive/MyDrive/ML/match_ids_1_unique.json", "rb") as f:
  matches = pickle.load(f)

matches

In [ ]:
matches = matches["match_ids"]

In [ ]:
def updateWinMatrix(champion_stats, data):
    # Process the data and update the statistics
    for entry in data:
        champion_name = entry[0]
        champion_id = entry[1]
        position = entry[2]
        win = entry[3]
        if position == '':
          print("No Position")
          continue

        # Initialize the dictionary if the champion is not present
        if champion_id not in champion_stats:
            champion_stats[champion_id] = {'TOP': {'wins': 0, 'losses': 0},
                                           'JUNGLE': {'wins': 0, 'losses': 0},
                                           'MIDDLE': {'wins': 0, 'losses': 0},
                                           'BOTTOM': {'wins': 0, 'losses': 0},
                                           'UTILITY': {'wins': 0, 'losses': 0}}

        # Update the statistics based on the win status
        if win:
            champion_stats[champion_id][position]['wins'] += 1
        else:
            champion_stats[champion_id][position]['losses'] += 1
    return champion_stats

In [ ]:
def updateSynergyMatrix(matrix, data, matrixSize, keyToIndex):
    # Process the data and update the matrix for the first 5 champions
    for i in range(5):
        for j in range(5):
            if i != j:  # Exclude the champion itself
                try:
                  win = data[i][3]
                  teammateKey = data[j][1]
                  championKey = data[i][1]
                  matrix[keyToIndex[championKey]][keyToIndex[teammateKey]]['wins' if win else 'losses'] += 1
                except:
                  continue

    # Process the data and update the matrix for the remaining 5 champions
    for i in range(5, 10):
        for j in range(5, 10):
            if i != j:  # Exclude the champion itself
                try:
                  win = data[i][3]
                  teammateKey = data[j][1]
                  championKey = data[i][1]
                  matrix[keyToIndex[championKey]][keyToIndex[teammateKey]]['wins' if win else 'losses'] += 1
                except:
                  continue
    return matrix

In [ ]:
def updateCounterMatrix(matrix, data, matrixSize, keyToIndex):
    for i in range(5):
      for j in range(5,10):
        try:
          if data[i][2] == data[j][2]:
            win = data[i][3]
            enemyKey = data[j][1]
            championKey = data[i][1]
            matrix[keyToIndex[championKey]][keyToIndex[enemyKey]]['wins' if win else 'losses'] += 1
            matrix[keyToIndex[enemyKey]][keyToIndex[championKey]]['losses' if win else 'wins'] += 1
        except:
          continue
    return matrix

In [ ]:
url = "https://ddragon.leagueoflegends.com/cdn/13.24.1/data/en_US/champion.json"
r = requests.get(url=url)
r = r.json()
keys = [int(championData["key"]) for championName, championData in r['data'].items()]

In [ ]:
participants = []
champion_stats = {}
#champion_to_index = {champion[1]: index for index, champion in enumerate(data)}
keyToIndex = {key: index for index, key in enumerate(keys)}
matrixSize = len(keys)
synergyMatrix = [[{'wins': 0, 'losses': 0} for _ in range(matrixSize)] for _ in range(matrixSize)]
counterMatrix = [[{'wins': 0, 'losses': 0} for _ in range(matrixSize)] for _ in range(matrixSize)]

In [ ]:
def checkpoint():
  with open("/content/drive/MyDrive/ML/winRateMatrix.pkl", 'wb') as file:
      pickle.dump(champion_stats, file)
  with open("/content/drive/MyDrive/ML/synergyMatrix.pkl", 'wb') as file:
      pickle.dump(synergyMatrix, file)
  with open("/content/drive/MyDrive/ML/counterMatrix.pkl", 'wb') as file:
      pickle.dump(counterMatrix, file)
  with open("/content/drive/MyDrive/ML/dataset.pkl", 'wb') as file:
      pickle.dump(participants, file)

In [ ]:
def getMastery(championId, puuid):
  url = f"https://euw1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}/by-champion/{championId}"
  r = requests.get(url=url, headers=headers)
  if(r.status_code == 200):
    r = r.json()
    return  r["championLevel"], r["championPoints"]
  else:
    print("Error getting mastery")
    return 0, 0

In [ ]:
def getChampions(r):
  champions = [[participant["championName"],participant["championId"],participant["teamPosition"], participant["win"]] for participant in r["info"]["participants"]]
  return champions

In [ ]:
def getMatchParticipants(r):
  participants = [[participant["summonerId"], participant["puuid"]] for participant in r["info"]["participants"]]
  return participants

In [ ]:
count = 0;
for matchId in matches:

  #Save to drive every 5000 games
  if count == 5000:
    checkpoint()
    count = 0

  url4 = f"https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}"
  r = requests.get(url=url4, headers=headers)
  time.sleep(0.8)

  if(r.status_code == 200):
    r = r.json()
    champions = getChampions(r)
    summoners = getMatchParticipants(r)

    #Updating matrices
    champion_stats = updateWinMatrix(champion_stats, champions)
    synergyMatrix = updateSynergyMatrix(synergyMatrix, champions, len(keys), keyToIndex)
    counterMatrix = updateCounterMatrix(counterMatrix, champions, len(keys), keyToIndex)

    #Storing Game data
    participant = []
    try:
      for x in range(10):
        masteryLevel, masteryPoints = getMastery(champions[x][1], summoners[x][1])
        time.sleep(0.8)
        data = {
            'summonerId': summoners[x][0],
            'puuid': summoners[x][1],
            'champion': champions[x][0],
            'championId': champions[x][1],
            'role': champions[x][2],
            'win': champions[x][3],
            'masteryLevel': masteryLevel,
            'masteryPoints': masteryPoints
        }
        participant.append(data)
    except:
        continue
  participants.append(participant)
  count+=1

### Feature Engineering

In [ ]:
with open("/content/drive/MyDrive/ML/winRateMatrix.pkl", 'rb') as file:
      champion_stats = pickle.load(file)
with open("/content/drive/MyDrive/ML/synergyMatrix.pkl", 'rb') as file:
      synergyMatrix = pickle.load(file)
with open("/content/drive/MyDrive/ML/counterMatrix.pkl", 'rb') as file:
      counterMatrix = pickle.load(file)
with open("/content/drive/MyDrive/ML/dataset.pkl", 'rb') as file:
      matches = pickle.load(file)

In [ ]:
totalWins = 0
totalLosses = 0

for champId, positions in champion_stats.items():
    for position, stats in positions.items():
        if stats['wins'] + stats['losses'] < 50:
            totalWins += stats['wins']
            totalLosses += stats['losses']
print(totalWins/ (totalWins + totalLosses))
avgAutofillRating = (totalWins - totalLosses)/ (totalWins + totalLosses)
avgAutofillRating

In [ ]:
def getMatchWinner(matchData):
  if(matchData[0]["win"] == True):
    return 0
  else:
    return 1

In [ ]:
def getWinLoss(champion_id, position_index):
    position_mapping = {0: 'TOP', 1: 'JUNGLE', 2: 'MIDDLE', 3: 'BOTTOM', 4: 'UTILITY'}

    if position_index not in position_mapping:
        raise ValueError("Invalid position index")

    position = position_mapping[position_index]

    # Get wins and losses for the specified champion and position
    stats = champion_stats[champion_id].get(position, {'wins': 0, 'losses': 0})
    wins = stats['wins']
    losses = stats['losses']

    if(wins+losses < 50):
      return avgAutofillRating
    else:
      return (wins - losses)/(wins + losses)

In [ ]:
def getSynergy(matchData):
    blueRating = 0
    redRating = 0
    for i in range(5):
      championKey = matchData[i]["championId"]
      for j in range(i+1, 5):
        teammateKey = matchData[j]["championId"]
        wins = synergyMatrix[keyToIndex[championKey]][keyToIndex[teammateKey]]["wins"]
        losses = synergyMatrix[keyToIndex[championKey]][keyToIndex[teammateKey]]["losses"]
        blueRating += ((wins - losses)/(wins + losses))

    for i in range(5, 10):
      championKey = matchData[i]["championId"]
      for j in range(i+1, 10):
        teammateKey = matchData[j]["championId"]
        wins = synergyMatrix[keyToIndex[championKey]][keyToIndex[teammateKey]]["wins"]
        losses = synergyMatrix[keyToIndex[championKey]][keyToIndex[teammateKey]]["losses"]
        redRating += ((wins - losses)/(wins + losses))

    return blueRating, redRating


In [ ]:
def getDeltas(matchData):
  deltas = []

  for i in range(5):
      enemyKey = matchData[i+5]["championId"]
      championKey = matchData[i]["championId"]
      wins = counterMatrix[keyToIndex[championKey]][keyToIndex[enemyKey]]["wins"]
      losses = counterMatrix[keyToIndex[championKey]][keyToIndex[enemyKey]]["losses"]
      if wins+losses == 0:
        deltas.append(0)
      else:
        deltas.append((wins-losses)/(wins+losses))

  return deltas[0], deltas[1], deltas[2], deltas[3], deltas[4]

In [ ]:
matches = matches[:10000]

In [ ]:
dataset = []
for matchData in matches:
  blueTotalMasteryLevel = 0
  blueTotalMasteryPoints = 0
  redTotalMasteryLevel = 0
  redTotalMasteryPoints = 0
  blueWinRatings = []
  redWinRatings = []

  teamWin = getMatchWinner(matchData)
  blueSynergyScore, redSynergyScore = getSynergy(matchData)

  for i, participant in enumerate(matchData):
    champId = participant["championId"]
    pos = i % 5
    if i < 5:
      blueTotalMasteryLevel += participant["masteryLevel"]
      blueTotalMasteryPoints += participant["masteryPoints"]
      blueWinRatings.append(getWinLoss(champId, pos))
    else:
      redTotalMasteryLevel += participant["masteryLevel"]
      redTotalMasteryPoints += participant["masteryPoints"]
      redWinRatings.append(getWinLoss(champId, pos))

  topDelta, jgDelta, midDelta, botDelta, supDelta = getDeltas(matchData)

  features = {
    'blueTopRating': blueWinRatings[0],
    'blueJgRating': blueWinRatings[1],
    'blueMidRating': blueWinRatings[2],
    'blueBotRating': blueWinRatings[3],
    'blueSupRating': blueWinRatings[4],
    'redTopRating': redWinRatings[0],
    'redJgRating': redWinRatings[1],
    'redMidRating': redWinRatings[2],
    'redBotRating': redWinRatings[3],
    'redSupRating': redWinRatings[4],
    'blueAvgMasteryLevel': blueTotalMasteryLevel / 5,
    'blueAvgMasteryPoints': blueTotalMasteryPoints / 5,
    'redAvgMasteryLevel': redTotalMasteryLevel / 5,
    'redAvgMasteryPoints': redTotalMasteryPoints / 5,
    'blueSynergyScore': blueSynergyScore,
    'redSynergyScore': redSynergyScore,
    'topDelta': topDelta,
    'jgDelta': jgDelta,
    'midDelta': midDelta,
    'botDelta': botDelta,
    'supDelta': supDelta,
    'teamWin': teamWin
  }
  dataset.append(features)

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)
df

In [ ]:
df.to_csv("/content/drive/MyDrive/ML/data.csv", index=False)

### EDA

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ML/data.csv")

In [ ]:
df['teamWin'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Convert the 'wins' and 'losses' to win rate and store it in a new matrix
winRateMatrix = np.zeros((matrixSize, matrixSize))
for i in range(matrixSize):
    for j in range(matrixSize):
        if synergyMatrix[i][j]['wins'] + synergyMatrix[i][j]['losses'] > 0:
            winRateMatrix[i][j] = (synergyMatrix[i][j]['wins'] - synergyMatrix[i][j]["losses"]) / (synergyMatrix[i][j]['wins'] + synergyMatrix[i][j]['losses'])

# Define the indices for the entire matrix (0 to 10)
startRow, endRow = 0, 10
startCol, endCol = 0, 10

# Select the entire matrix
subsetMatrix = winRateMatrix[startRow:endRow, startCol:endCol]

# Plot the entire matrix with axes on the left and bottom
fig, ax = plt.subplots()
cax = ax.matshow(subsetMatrix, cmap='viridis')

# Display the actual values in each cell for the subset
for i in range(subsetMatrix.shape[0]):
    for j in range(subsetMatrix.shape[1]):
        value = subsetMatrix[i, j]
        ax.text(j, i, f'{value:.1f}', va='center', ha='center', color='white' if value < 0.5 else 'black')

# Set the ticks and labels on the left and bottom for the subset
ax.set_xticks(np.arange(startCol, endCol))
ax.set_yticks(np.arange(startRow, endRow))
ax.set_xticklabels(keys[startCol:endCol])
ax.set_yticklabels(keys[startRow:endRow])

# Add colorbar
cbar = fig.colorbar(cax)
cbar.set_label('Win Rate')

# Small snippet
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Convert the 'wins' and 'losses' to win rate and store it in a new matrix
winRateMatrix = np.zeros((matrixSize, matrixSize))
for i in range(matrixSize):
    for j in range(matrixSize):
        if counterMatrix[i][j]['wins'] + counterMatrix[i][j]['losses'] > 0:
            winRateMatrix[i][j] = (counterMatrix[i][j]['wins'] - counterMatrix[i][j]["losses"]) / (counterMatrix[i][j]['wins'] + counterMatrix[i][j]['losses'])

# Define the indices for the entire matrix (0 to 10)
startRow, endRow = 0, 10
startCol, endCol = 0, 10

# Select the entire matrix
subsetMatrix = winRateMatrix[startRow:endRow, startCol:endCol]

# Plot the entire matrix with axes on the left and bottom
fig, ax = plt.subplots()
cax = ax.matshow(subsetMatrix, cmap='viridis')

# Display the actual values in each cell for the subset
for i in range(subsetMatrix.shape[0]):
    for j in range(subsetMatrix.shape[1]):
        value = subsetMatrix[i, j]
        ax.text(j, i, f'{value:.1f}', va='center', ha='center', color='white' if value < 0.5 else 'black')

# Set the ticks and labels on the left and bottom for the subset
ax.set_xticks(np.arange(startCol, endCol))
ax.set_yticks(np.arange(startRow, endRow))
ax.set_xticklabels(keys[startCol:endCol])
ax.set_yticklabels(keys[startRow:endRow])

# Add colorbar
cbar = fig.colorbar(cax)
cbar.set_label('Win Rate')

# Small snippet
plt.show()